# Compilation

---

## Automate schema 
i made a dynamic way that allows me to create manopulate the automathe without change the code if the lexical analizer


In [53]:

import string

final_states = {
    9:  "Keyword_programme",
    18: "Keyword_constante",
    34: "Keyword_variable",
    39: "Keyword_debut",
    42: "Keyword_fin",
    49: "Keyword_répéter",
    58: "Keyword_jusqua",
    
    26: "TypeName_caractere",
    51: "TypeName_réel",
    64: "TypeName_entier",
    
    65: "MultOperator_et",
    68: "MultOperator_mod",
    71: "MultOperator_div",
    69: "MultOperator_mult",
    
    73: "AddOperator_ou",
    78: "AddOperator_plus",
    79: "AddOperator_minus",
    80: "RelOperator_L",
    81: "RelOperator_LE",
    82: "RelOperator_NE",
    83: "RelOperator_G",
    84: "RelOperator_GE",
    85: "RelOperator_E",
    
    86: "Semicolon",
    
    75: "Name",
    
    77: "Number",
    
    88: "Float",
    
    89: "END",
    90: "COMMA",
    91: "COLON",
    92: "ASSIGNMENT",
    
    93: "left-parenthesis",
    94: "right-parenthesis",
    
    
}


# the states should be referred only by numbers 
rules = {
    0: {
        # Keywords
        "p": 1, "c": 10, "v": 27, "d": 35, "f": 40, "r": 43, "j": 52, ";": 86,
        ".": 89, ",": 90, ":": 91,
        
        # Relational Operators
        "<": 80, ">": 83, "=": 85,
        
        # Addition Operators
        "+": 78, 
        "-": 79,  
        "o": 72,
        
        # Multiplication Operators
        "e": 59, "m": 66, "*": 69, 
        
        # Parenthesis
        "(": 93, ")": 94,
        
        # Name
        **{x: 74 for x in string.ascii_letters if x not in "pcvdfrjome"},
        
        # Numbers
        **{x: 77 for x in string.digits},
    },
    
    # Keywords
    1: {"r": 2,**{x: 75 for x in string.digits}},
    2: {"o": 3},
    3: {"g": 4},
    4: {"r": 5},
    5: {"a": 6},
    6: {"m": 7},
    7: {"m": 8},
    8: {"e": 9},  # programme_FOUND
    
    10: {"o": 11, "a": 19,**{x: 75 for x in string.digits}},
    11: {"n": 12},
    12: {"s": 13},
    13: {"t": 14},
    14: {"a": 15},
    15: {"n": 16},
    16: {"t": 17},
    17: {"e": 18},  # constante_FOUND
    
    19: {"r": 20},
    20: {"a": 21},
    21: {"c": 22},
    22: {"t": 23},
    23: {"è": 24,"e":24},
    24: {"r": 25},
    25: {"e": 26},  # caractère_FOUND
    
    27: {"a": 28,**{x: 75 for x in string.digits}},
    28: {"r": 29},
    29: {"i": 30},
    30: {"a": 31},
    31: {"b": 32},
    32: {"l": 33},
    33: {"e": 34},  # variable_FOUND
    
    35: {"e": 36, "i": 70,**{x: 75 for x in string.digits}},
    36: {"b": 37},
    37: {"u": 38},
    38: {"t": 39},  # debut_FOUND
    70: {"v": 71},
    
    40: {"i": 41,**{x: 75 for x in string.digits}},
    41: {"n": 42},  # fin_FOUND
    
    43: {"é": 44, "e": 44,**{x: 75 for x in string.digits}},
    44: {"p": 45, "é": 50, "e": 50},
    45: {"é": 46, "e": 46},
    46: {"t": 47},
    47: {"e": 48},
    48: {"r": 48},  # répéter_FOUND
    
    50: {"l": 51},  # reel_FOUND
    
    52: {"u": 53,**{x: 75 for x in string.digits}},
    53: {"s": 54},
    54: {"q": 55},
    55: {"u": 56},
    56: {"'": 57},
    57: {"a": 58, "à": 58},  # jusqua_FOUND
    
    59: {"n": 60, "t": 65,**{x: 75 for x in string.digits}},
    60: {"t": 61},
    61: {"i": 62},
    62: {"e": 63},
    63: {"r": 64},  # entier_FOUND
    
    66: {"o": 67,**{x: 75 for x in string.digits}},
    67: {"d": 68},  # mod_FOUND
    
    72: {"u": 73,**{x: 75 for x in string.digits}},  # ou_FOUND
    
    87 :{**{x: 88 for x in string.digits}},
    
    
    
    # Final states
    **{i: {} for i in final_states.keys()},
    
    # overide the final states with other states
    74: {**{x: 75 for x in string.digits}},
    75: {**{x: 76 for x in string.ascii_lowercase}},
    76: {**{x: 75 for x in string.digits}},
    77: {**{x: 77 for x in string.digits}, ".": 87},
    80: {">": 82, "=": 81},
    83: {"=": 84},
    88: {**{x: 88 for x in string.digits}},
    91: {"=": 92},
}


---

## 2. Lexical Analizer

In [170]:
class LexicalAnalizer:
    def __init__(self, rules, final_states):
        self.rules = rules
        self.final_states = final_states

    def run(self, code):
        code = code.lower()
        tokens = []
        state = 0
        buffer = ""
        position = 0

        while position < len(code):
            char = code[position]
            next_state = self.rules.get(state, {}).get(char)
            if char.isspace():
                if state in self.final_states:
                    tokens.append((buffer, self.final_states[state], position - len(buffer)))
                state = 0
                buffer = ""
            elif next_state is not None:
                state = next_state
                buffer += char

            else:
                if state in self.final_states:
                    tokens.append((buffer, self.final_states[state], position - len(buffer)))
                    state, buffer = 0, ""
                    if char != " ":
                        position -= 1
                elif state != 0:
                    raise Exception(f"Lexical Error: Unexpected character '{char}' after `{buffer}` at position {position}")
                else:
                    raise Exception(f"Lexical Error: Unexpected character '{char}' at position {position}")

            position += 1
        if state in self.final_states:
            tokens.append((buffer, self.final_states[state], position - len(buffer)))
        return tokens

lexer = LexicalAnalizer(rules, final_states)
tokens = lexer.run("programme a1; variable b1; constante c1; debut répéter jusqu'a; fin.")
print(tokens)

[('programme', 'Keyword_programme', 0), ('a1', 'Name', 10), (';', 'Semicolon', 12), ('variable', 'Keyword_variable', 14), ('b1', 'Name', 23), (';', 'Semicolon', 25), ('constante', 'Keyword_constante', 27), ('c1', 'Name', 37), (';', 'Semicolon', 39), ('debut', 'Keyword_debut', 41), ("jusqu'a", 'Keyword_jusqua', 55), (';', 'Semicolon', 62), ('fin', 'Keyword_fin', 64), ('.', 'END', 67)]


In [25]:

keywords = ["programme;", "<\nconstante12x3", "variable", "debut", "fin", "répéter", "jusqu'à", "div", "mod", "et", "ou", "entier", "caractère", "réel"]
text = " ".join(keywords)
print(text)
tokens = lex(text)
print(tokens)

programme; <
constante12x3 variable debut fin répéter jusqu'à div mod et ou entier caractère réel
> Loop 0 =======================================================
char: p, state: 0, next_state: 1, buffer: ``
    > char is not space and next state = 1 (not None)
    > buffer: `p`
> Loop 1 =======================================================
char: r, state: 1, next_state: 2, buffer: `p`
    > char is not space and next state = 2 (not None)
    > buffer: `pr`
> Loop 2 =======================================================
char: o, state: 2, next_state: 3, buffer: `pr`
    > char is not space and next state = 3 (not None)
    > buffer: `pro`
> Loop 3 =======================================================
char: g, state: 3, next_state: 4, buffer: `pro`
    > char is not space and next state = 4 (not None)
    > buffer: `prog`
> Loop 4 =======================================================
char: r, state: 4, next_state: 5, buffer: `prog`
    > char is not space and next state = 5 (not 

---

## 3. Syntaxic

In [115]:
grammar = {
    "<ProgrammeAlgoLang>": [final_states[9], final_states[75], final_states[86], "<Corps>", final_states[89]],
    "<Corps>": ["[<PartieDéfinitionConstante>]", "[<PartieDéfinitionVariable>]", "<InstrComp>"],
    "<PartieDéfinitionConstante>": ["constante", "<DéfinitionConstante>", "{<DéfinitionConstante>}"],
    "<DéfinitionConstante>": [final_states[75], "=", "<Constante>", final_states[86]],
    "<PartieDéfinitionVariable>": ["variable", "<DéfinitionVariable>", "{<DéfinitionVariable>}"],
    "<DéfinitionVariable>": ["<GroupeVariable>", final_states[86]],
    "<GroupeVariable>": [final_states[75], "{<MultiVars>}", final_states[91], final_states[26].split("_")[0]],
    "<MultiVars>": [final_states[90] ,final_states[75]], # new
    "<Constante>": [
        [final_states[77]], 
        [final_states[75]]
    ],
    "<InstrComp>": [final_states[39], "<Instruction>","{<MultiInstcations>}",final_states[42]],
    "<MultiInstcations>":  [final_states[86], "<Instruction>"], # new
    "<Instruction>": [
        ["<InstructionAffectation>"], 
        ["<InstructionRépéter>"], 
        ["<InstrComp>"], 
        ["<Vide>"]
    ],
    "<InstructionAffectation>": [final_states[75], ":=", "<Expression>"],
    "<Expression>": ["<ExpressionSimple>", "[<OpRelExp>]"],
    "<OpRelExp>": [final_states[80].split("_")[0], "<ExpressionSimple>"], #new
    "<ExpressionSimple>": [f"[<{final_states[78].split("_")[0]}>]", "<Terme>","[<OpAdTerm>]"],
    "<OpAdTerm>":  [final_states[73].split("_")[0], "<Terme>"], #new
    # "<OperateurSigne>": ["+", "-"], # tobe fixed
    "<Terme>": ["<Facteur>", "[<OpMulFact>]"],
    "<OpMulFact>": [final_states[65].split("_")[0], "<Facteur>"], #new
    "<Facteur>": [
        ["<Constante>"], 
        [final_states[75]], 
        [final_states[93], "<Expression>", final_states[94]]
    ],
    "<InstructionRépéter>": [final_states[49], "<Instruction>", final_states[58], "<Condition>"],
    "<Condition>": [
        ["<Expression>", final_states[80].split("_")[0], "<Expression>"], 
        [final_states[93],"<Condition>",final_states[94]] 
    ],
    "<Vide>": [""]
}
print(grammar)

{'<ProgrammeAlgoLang>': ['Keyword_programme', 'Name', 'Semicolon', '<Corps>', 'END'], '<Corps>': ['[<PartieDéfinitionConstante>]', '[<PartieDéfinitionVariable>]', '<InstrComp>'], '<PartieDéfinitionConstante>': ['constante', '<DéfinitionConstante>', '{<DéfinitionConstante>}'], '<DéfinitionConstante>': ['Name', '=', '<Constante>', 'Semicolon'], '<PartieDéfinitionVariable>': ['variable', '<DéfinitionVariable>', '{<DéfinitionVariable>}'], '<DéfinitionVariable>': ['<GroupeVariable>', 'Semicolon'], '<GroupeVariable>': ['Name', '{<MultiVars>}', 'COLON', 'TypeName'], '<MultiVars>': ['COMMA', 'Name'], '<Constante>': [['Number'], ['Name']], '<InstrComp>': ['Keyword_debut', '<Instruction>', '{<MultiInstcations>}', 'Keyword_fin'], '<MultiInstcations>': ['Semicolon', '<Instruction>'], '<Instruction>': [['<InstructionAffectation>'], ['<InstructionRépéter>'], ['<InstrComp>'], ['<Vide>']], '<InstructionAffectation>': ['Name', ':=', '<Expression>'], '<Expression>': ['<ExpressionSimple>', '[<OpRelExp>]'

In [152]:
class SyntaxAnalyzer:
    def __init__(self, tokens):
        self.tokens = tokens
        self.current_token_index = 0

    def match(self, expected_token_type):
        if self.current_token_index < len(self.tokens) and self.tokens[self.current_token_index][1] == expected_token_type:
            self.current_token_index += 1
        else:
            raise SyntaxError(f"Expected {expected_token_type}, found {self.tokens[self.current_token_index][1]}")

    def parse_ProgrammeAlgoLang(self):
        self.match("Keyword_programme")
        self.match("Name")
        self.match("Semicolon")
        self.parse_Corps()
        self.match("END")

    def parse_Corps(self):
        if self.current_token_index < len(self.tokens) and self.tokens[self.current_token_index][1] == "Keyword_constante":
            self.parse_PartieDéfinitionConstante()
        if self.current_token_index < len(self.tokens) and self.tokens[self.current_token_index][1] == "Keyword_variable":
            self.parse_PartieDéfinitionVariable()
        self.parse_InstrComp()

    def parse_PartieDéfinitionConstante(self):
        self.match("Keyword_constante")
        self.parse_DéfinitionConstante()
        while self.current_token_index < len(self.tokens) and self.tokens[self.current_token_index][1] == "Name":
            self.parse_DéfinitionConstante()

    def parse_DéfinitionConstante(self):
        self.match("Name")
        self.match("=")
        self.parse_Constante()
        self.match("Semicolon")

    def parse_PartieDéfinitionVariable(self):
        self.match("Keyword_variable")
        self.parse_DéfinitionVariable()
        while self.current_token_index < len(self.tokens) and self.tokens[self.current_token_index][1] == "Name":
            self.parse_DéfinitionVariable()

    def parse_DéfinitionVariable(self):
        self.parse_GroupeVariable()
        self.match("Semicolon")

    def parse_GroupeVariable(self):
        self.match("Name")
        while self.current_token_index < len(self.tokens) and self.tokens[self.current_token_index][1] == "COMMA":
            self.match("COMMA")
            self.match("Name")
        self.match("COLON")
        self.match("TypeName_entier")

    def parse_Constante(self):
        if self.current_token_index < len(self.tokens) and self.tokens[self.current_token_index][1] == "Number":
            self.match("Number")
        else:
            self.match("Name")

    def parse_InstrComp(self):
        self.match("Keyword_debut")
        self.parse_Instruction()
        while self.current_token_index < len(self.tokens) and self.tokens[self.current_token_index][1] == "Semicolon":
            self.match("Semicolon")
            self.parse_Instruction()
        self.match("Keyword_fin")

    def parse_Instruction(self):
        if self.current_token_index < len(self.tokens) and self.tokens[self.current_token_index][1] == "Name":
            self.parse_InstructionAffectation()
        elif self.current_token_index < len(self.tokens) and self.tokens[self.current_token_index][1] == "Keyword_répéter":
            self.parse_InstructionRépéter()
        elif self.current_token_index < len(self.tokens) and self.tokens[self.current_token_index][1] == "Keyword_debut":
            self.parse_InstrComp()
        else:
            self.parse_Vide()

    def parse_InstructionAffectation(self):
        self.match("Name")
        self.match("ASSIGNMENT")
        self.parse_Expression()

    def parse_Expression(self):
        self.parse_ExpressionSimple()
        if self.current_token_index < len(self.tokens) and self.tokens[self.current_token_index][1] == "RelOperator_L":
            self.parse_OpRelExp()

    def parse_OpRelExp(self):
        self.match("RelOperator_L")
        self.parse_ExpressionSimple()

    def parse_ExpressionSimple(self):
        if self.current_token_index < len(self.tokens) and self.tokens[self.current_token_index][1] == "AddOperator_plus":
            self.match("AddOperator_plus")
        self.parse_Terme()
        if self.current_token_index < len(self.tokens) and self.tokens[self.current_token_index][1] == "AddOperator_ou":
            self.parse_OpAdTerm()

    def parse_OpAdTerm(self):
        self.match("AddOperator_ou")
        self.parse_Terme()

    def parse_Terme(self):
        self.parse_Facteur()
        if self.current_token_index < len(self.tokens) and self.tokens[self.current_token_index][1] == "MultOperator_et":
            self.parse_OpMulFact()

    def parse_OpMulFact(self):
        self.match("MultOperator_et")
        self.parse_Facteur()

    def parse_Facteur(self):
        if self.current_token_index < len(self.tokens) and self.tokens[self.current_token_index][1] == "Number":
            self.parse_Constante()
        elif self.current_token_index < len(self.tokens) and self.tokens[self.current_token_index][1] == "Name":
            self.match("Name")
        else:
            self.match("left-parenthesis")
            self.parse_Expression()
            self.match("right-parenthesis")

    def parse_InstructionRépéter(self):
        self.match("Keyword_répéter")
        self.parse_Instruction()
        self.match("Keyword_jusqua")
        self.parse_Condition()

    def parse_Condition(self):
        self.parse_Expression()
        self.match("RelOperator_L")
        self.parse_Expression()

    def parse_Vide(self):
        pass

# Example usage
analyzer = SyntaxAnalyzer(lex("programme x1; debut x2 := 2; fin."))
analyzer.parse_ProgrammeAlgoLang()
print("Parsing successful!")

Parsing successful!


In [20]:
bakup_grammar = {
    "<ProgrammeAlgoLang>": [["programme", "<NomProgramme>", ";", "<Corps>", "."]],
    "<Corps>": [["<PartieDéfinitionConstante>", "<PartieDéfinitionVariable>", "<InstrComp>"]],
    "<PartieDéfinitionConstante>": [["constante", "<DéfinitionConstante>", "{", "<DéfinitionConstante>", "}"]],
    "<DéfinitionConstante>": [["<NomConstante>", "=", "<Constante>", ";"]],
    "<PartieDéfinitionVariable>": [["variable", "<DéfinitionVariable>", "{", "<DéfinitionVariable>", "}"]],
    "<DéfinitionVariable>": [["<GroupeVariable>", ";"]],
    "<GroupeVariable>": [["<NomVariable>", "{", ",", "<NomVariable>", "}", ":", "<NomType>"]],
    "<NomVariable>": [["<Nom>"]],
    "<Nom>": [["<Lettre>", "<Chiffre>", "{", "<Lettre>", "<Chiffre>", "}"]],
    "<Lettre>": [[x] for x in string.ascii_uppercase+string.ascii_lowercase+"éèà"] ,
    "<Chiffre>": [[f"x"] for x in string.digits],
    "<NomType>": [["Entier"], ["Caractère"], ["Réel"]],
    "<NomConstante>": [["<Nom>"]],
    "<Constante>": [["<Nombre>"], ["<NomConstante>"]],
    "<NomProgramme>": [["<Nom>"]],
    "<Nombre>": [["<Chiffre>", "{", "<Chiffre>", "}"]],
    "<InstrComp>": [["debut", "<Instruction>", "{", ";", "<Instruction>", "}", "fin"]],
    "<Instruction>": [["<InstructionAffectation>"], ["<InstructionRépéter>"], ["<InstrComp>"], ["<Vide>"]],
    "<InstructionAffectation>": [["<NomVariable>", ":=", "<Expression>"]],
    "<Expression>": [["<ExpressionSimple>", "[", "<OperateurRelationnel>", "<ExpressionSimple>", "]"]],
    "<OperateurRelationnel>": [["<"], [">"], ["="], ["<="], [">="], ["<>"]],
    "<ExpressionSimple>": [["[", "<OperateurSigne>", "]", "<Terme>", "{", "<OperateurAddition>", "<Terme>", "}"]],
    "<OperateurSigne>": [["+"], ["-"]],
    "<OperateurAddition>": [["+"], ["-"], ["ou"]],
    "<Terme>": [["<Facteur>", "{", "<OperateurMult>", "<Facteur>", "}"]],
    "<OperateurMult>": [["*"], ["div"], ["mod"], ["et"]],
    "<Facteur>": [["<Constante>"], ["<NomVariable>"], ["(", "<Expression>", ")"]],
    "<InstructionRépéter>": [["répéter", "<Instruction>", "jusqu’à", "<Condition>"]],
    "<Condition>": [["<Expression>", "<OperateurRelationnel>", "<Expression>"], ["(", "<Condition>", ")"]],
    "<Vide>": [[""]]
}

plan x